In [1]:
# %matplotlib widget
import os
import sys
# import netCDF4 as nc
import warnings

import cartopy.crs as ccrs
import cartopy.feature as cfeat
import matplotlib.pyplot as plt
import numpy as np
import pymannkendall as mk
import xarray as xr
from cartopy.io.shapereader import Reader
from cartopy.mpl.gridliner import LATITUDE_FORMATTER, LONGITUDE_FORMATTER
from geocat.viz import util as gvutil

warnings.filterwarnings("ignore")

In [2]:
# 设置相对路径
data_dir = os.path.join(os.path.pardir, "/mnt/e/tian/data")
print("This notebook was built with:")
print("python version: %s " % sys.version)
print(data_dir)

This notebook was built with:
python version: 3.10.11 | packaged by conda-forge | (main, May 10 2023, 18:58:44) [GCC 11.3.0] 
/mnt/e/tian/data


# 1.读取文件并创建写出变量
## 1）读取netcdf文件

In [3]:
# 读取文件
var_name = "deltaGWI"
file = os.path.join(data_dir, "GWIdelta_yearmean_2002-2020.nc")
# print(file)
f = xr.open_dataset(file)
var = f.deltaGWI
# print(var)
dim_1 = var.shape[1]
dim_2 = var.shape[2]
# print(dim_1, dim_2)
lat = f.lat
lon = f.lon
lats = f.lat.values
lons = f.lon.values
# 设置输出及其相关变量属性
slope_matrix = f.deltaGWI[0, :, :]
p_matrix = f.deltaGWI[0, :, :]
tau_matrix = f.deltaGWI[0, :, :]
# print(slope_matrix)
slope_matrix.attrs['long_name'] = 'Theil-Sen estimator/slope'
# slope_matrix.attrs['units'] = 'mm/10yr'
p_matrix.attrs['long_name'] = 'p-value of the significance test'
p_matrix.attrs['units'] = ''
tau_matrix.attrs['long_name'] = 'Kendall Tau'
tau_matrix.attrs['units'] = ''
# print(slope_matrix)

## 2）Mann-kendall test

In [ ]:
# Mann-Kendall test
for x in range(dim_1):
    for y in range(dim_2):
        var_list = np.array(var[:, x, y])
        if np.isnan(var_list).any():
            slope_matrix[x, y] = np.nan
            p_matrix[x, y] = np.nan
            tau_matrix[x, y] = np.nan
        else:
            mkrslt = mk.trend_free_pre_whitening_modification_test(var_list)
            slope_matrix[x, y] = 10*mkrslt.slope
            p_matrix[x, y] = mkrslt.p
            tau_matrix[x, y] = mkrslt.Tau


In [ ]:
# slope_nan = slope_matrix.copy()
# slope_nan[slope_nan == -9999] = np.nan
print(slope_matrix)
# print(np.nanmax(slope_matrix.values))
# print(np.nanmin(slope_matrix.values))

In [ ]:
slope_matrix

## 写出Mann-kendall检验结果

In [ ]:
# 写出计算结果到netcdf文件
# print(slope_matrix)
out = xr.Dataset({
    "slope": slope_matrix,
    "p": p_matrix,
    "tau": tau_matrix
})
# out.coords
del out['time']
# print(out)
out.attrs['Author'] = 'ninglk@igsnrr.ac.cn'
# out.drop_dims("time")
out.to_netcdf("../deltaGWI_tibet_mk.nc")

In [ ]:
# 设置显著性
lons, lats = np.meshgrid(p_matrix.lon, p_matrix.lat)
sig_area = np.where(p_matrix <= 0.05)
# 添加shapefile边界
reader = Reader(os.path.join(data_dir, "yzb.shp"))
proj = ccrs.PlateCarree()
feature = cfeat.ShapelyFeature(reader.geometries(), proj,
                               edgecolor='k', facecolor='none')

In [ ]:
# Generate figure (set its size (width, height) in inches)
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False
fig = plt.figure(figsize=(11, 6))
# %matplotlib widget
# Generate axes, using Cartopy
projection = ccrs.PlateCarree()
ax = plt.axes(projection=projection)

# Use global map and draw coastlines
# ax.set_global()
# ax.coastlines()

# Import an NCL colormap
# necmp = gvcmaps.BlueYellowRed_r
# print(necmp)
# cmap = plt.get_cmap('RdBu_r')
# newmap = gvutil.truncate_colormap(cmap, minval=0.3, maxval=0.9)
newcmp = gvutil.truncate_colormap(gvcmaps.BlueYellowRed_r,
                                  minval=0.20,
                                  maxval=1.0)
# Contourf-plot data (for filled contour s)
# Note, min-max contour levels are hard-coded. contourf's automatic contour value selector produces fractional values.
p = slope_matrix.plot.contourf(ax=ax,
                               vmin=-50,
                               vmax=80,
                               levels=14,
                               cmap=newcmp,
                               add_colorbar=False,
                               transform=projection,
                               extend='both',
                               extendfrac='auto')
p.ax.add_feature(feature, linewidth=1)
_ = ax.scatter(lons[sig_area], lats[sig_area], marker='.',
               s=5, c='k', alpha=1.0, transform=ccrs.PlateCarree())


# Add horizontal colorbar
# position=fig.add_axes([0.95, 0.24, 0.03, 0.84])
cbar = plt.colorbar(p, orientation='vertical', shrink=0.6)
cbar.ax.tick_params(labelsize=12)
cbar.set_label(slope_matrix.long_name + " (" + slope_matrix.units + ")")
cbar.set_ticks(np.linspace(-40, 80, 7))

# cbar.set_ticks(np.linspace(-80, 80, 17))

# Use geocat.viz.util convenience function to set axes tick values
gvutil.set_axes_limits_and_ticks(ax,
                                 xlim=(89.5, 123),
                                 ylim=(23, 37),
                                 xticks=np.linspace(90, 120, 7),
                                 yticks=np.linspace(24, 36, 7))

# Use geocat.viz.util convenience function to make plots look like NCL plots by using latitude, longitude tick labels
gvutil.add_lat_lon_ticklabels(ax)

# Use geocat.viz.util convenience function to add minor and major tick lines
gvutil.add_major_minor_ticks(ax,
                             x_minor_per_major=5,
                             y_minor_per_major=2,
                             labelsize=12)

# Use geocat.viz.util convenience function to add titles to left and right of the plot axis.
gvutil.set_titles_and_labels(ax,
                             maintitle="",
                             #                              maintitlefontsize=14,
                             #                              lefttitle=slope_matrix.long_name,
                             #                              lefttitlefontsize=12,
                             #                              righttitle=slope_matrix.units,
                             #                              righttitlefontsize=12,
                             xlabel="",
                             ylabel="")

# Show the plot
plt.savefig("yangtzepremk2.svg",  transparent=True,
            dpi=600, pad_inches=0.1, bbox_inches='tight')
plt.show()